# 코로나 전후 상권 변화 분석 - (1) 데이터 전처리

## #01. 패키지 참조

In [244]:
import numpy as np
from pandas import read_csv, concat, merge, DataFrame
from matplotlib import pyplot as plt
import seaborn as sb

## #02. 데이터 구축

### 1. 데이터 파일 열기

In [245]:
df2018 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2018.csv', encoding='euc-kr');
df2019 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2019.csv', encoding='euc-kr');
df2020 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2020.csv', encoding='euc-kr');
df2021 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권-추정매출)_2021.csv', encoding='euc-kr');
df상권영역 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권영역).csv', encoding='euc-kr');
df건축물 = read_csv('data/서울특별시 건축물대장 법정동 코드정보.csv', encoding='euc-kr');
df직장인구2018 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-직장인구)-2018.csv', encoding='euc-kr')
df직장인구2019 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-직장인구)-2019.csv', encoding='euc-kr')
df직장인구2020 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-직장인구)-2020.csv', encoding='euc-kr')
df직장인구2021 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-직장인구)-2021.csv', encoding='euc-kr')
df생활인구2018 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구)-2018.csv', encoding='euc-kr')
df생활인구2019 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구)-2019.csv', encoding='euc-kr')
df생활인구2020 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구)-2020.csv', encoding='euc-kr')
df생활인구2021 = read_csv('data/서울시 우리마을가게 상권분석서비스(상권배후지-생활인구)-2021.csv', encoding='euc-kr')

### 2. 상권영역 데이터 구축하기

#### 상권영역 데이터 확인

서울시에만 해당하는 데이터

In [246]:
df상권영역

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN
...,...,...,...,...,...,...,...,...,...,...
1491,201810,D,발달상권,1001044,양재역_3,202755,443008,11650,11650520,NaN
1492,201810,D,발달상권,1001030,양재역_2,203216,442691,11650,11650651,NaN
1493,201810,D,발달상권,1001031,서울 관악구 신림역_1,193706,442615,11620,11620645,NaN
1494,201810,D,발달상권,1001032,서울 관악구 신림역_2,193880,442602,11620,11620645,NaN


#### 법정동 코드정보 확인

In [247]:
df건축물['시군구코드'].value_counts()

시군구코드
47780    970
47740    921
47800    875
47810    835
43710    818
        ... 
27145      1
28265      1
25000      1
45118      1
22235      1
Name: count, Length: 500, dtype: int64

> `47780`이라는 코드를 갖는 지역이 `970`곳이 됨을 확인할 수 있다. 즉, 코드값의 중복이 있다.

#### 시군구명, 시군구코드를 추출하여 중복데이터 제거

In [248]:
df시군구 = df건축물.filter(['시군구명', '시군구코드'])
df시군구 = df시군구.drop_duplicates()
df시군구.dropna(inplace=True)
df시군구

,시군구명,시군구코드
0,영월군,42750
14,유성구,30200
23,평택시,41220
30,용인시 수지구,41465
32,용인시 처인구,41461
...,...,...
82870,대천시,44170
83192,온양시,44190
86860,동구 안심출장소,22145
88397,동해출장소,42105


#### 시군구명과 시군구코드를 상권영역에 결합

In [249]:
df상권영역2 = merge(df상권영역, df시군구, left_on='시군구_코드', right_on='시군구코드', how='left')
df상권영역2

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN,관악구,11620
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN,강남구,11680
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN,강남구,11680
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN,강서구,11500
...,...,...,...,...,...,...,...,...,...,...,...,...
1491,201810,D,발달상권,1001044,양재역_3,202755,443008,11650,11650520,NaN,서초구,11650
1492,201810,D,발달상권,1001030,양재역_2,203216,442691,11650,11650651,NaN,서초구,11650
1493,201810,D,발달상권,1001031,서울 관악구 신림역_1,193706,442615,11620,11620645,NaN,관악구,11620
1494,201810,D,발달상권,1001032,서울 관악구 신림역_2,193880,442602,11620,11620645,NaN,관악구,11620


#### `구`에 대한 정보 확인

##### 얼마나 많은 `구`가 있는가?

In [250]:
df상권영역2['시군구명'].value_counts()

시군구명
강남구     115
관악구      88
동대문구     79
영등포구     74
마포구      71
서초구      70
성북구      69
강서구      69
동작구      65
양천구      63
종로구      63
은평구      61
강동구      60
광진구      60
구로구      59
용산구      58
중구       55
강북구      52
성동구      48
중랑구      48
송파구      45
금천구      41
도봉구      33
서대문구     32
노원구      18
Name: count, dtype: int64

#### 파생변수 생성

`행정구`는 값의 종류가 너무 많아서 더미변수화 하기가 어려울 것으로 보고 도심과 비도심으로 구분하는 파생변수를 생성

> 출처: 서울2030플렌

In [251]:
도심 = ['종로구', '중구', '영등포구', '서초구', '강남구'] 

df상권영역2['도심'] = np.isin(df상권영역2['시군구명'], 도심)

df상권영역2.head()

,기준_년월_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,형태정보,시군구명,시군구코드,도심
0,201810,R,전통시장,1001453,낙성대시장,196121,442084,11620,11620585,NaN,관악구,11620,False
1,201810,R,전통시장,1001454,봉천제일종합시장,195147,442413,11620,11620595,NaN,관악구,11620,False
2,201810,R,전통시장,1001474,도곡시장,204551,444227,11680,11680650,NaN,강남구,11680,True
3,201810,R,전통시장,1001475,강남개포시장,206065,443310,11680,11680670,NaN,강남구,11680,True
4,201810,R,전통시장,1001412,화곡본동시장,186203,449328,11500,11500590,NaN,강서구,11500,False


#### 상권영역 데이터 저장

In [252]:
df상권영역2.to_excel('상권영역.xlsx', index=False)

### 3. 코로나 전후 매출데이터

#### 1) 코로나 이전

##### 단순 데이터 결합

In [253]:
코로나이전df = concat([df2018, df2019])
코로나이전df.reset_index(drop=True, inplace=True)
코로나이전df

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2018,1,D,발달상권,1001046,서울 관악구 신림역_4,CS200036,고시원,83495157,281,...,0,281,0,0,0,93,93,0,95,0
1,2018,1,D,발달상권,1001025,서울 관악구 서울대입구역_1,CS200012,법무사사무소,6571318,116,...,0,77,39,0,39,77,0,0,0,10
2,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,...,0,23,0,0,0,0,8,7,8,4
3,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,...,0,0,36,0,0,0,13,23,0,1
4,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,...,3,0,15,0,0,11,0,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298239,2019,4,R,전통시장,1001469,양재시장,CS300011,일반의류,14282341,48,...,0,0,48,0,0,48,0,0,0,2
298240,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,...,228,12935,9718,469,6166,6432,4792,3474,1319,2
298241,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,...,17654,57240,24608,2813,30887,15345,14338,12869,5596,2
298242,2019,4,D,발달상권,1001044,양재역_3,CS200001,일반교습학원,15509601,56,...,6,25,31,0,0,12,44,0,0,6


##### 골목상권 데이터만 추출

In [254]:
코로나이전df2 = 코로나이전df.query("상권_구분_코드_명 == '골목상권'")
코로나이전df2.head()

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2018,1,A,골목상권,1000289,인촌로17가길,CS300021,문구,6044732,23,...,0,23,0,0,0,0,8,7,8,4
3,2018,1,A,골목상권,1000515,남부순환로70길,CS200030,피부관리실,11863957,40,...,0,0,36,0,0,0,13,23,0,1
4,2018,1,A,골목상권,1000902,논현로63길,CS200029,네일숍,1766586,19,...,3,0,15,0,0,11,0,0,5,2
5,2018,1,A,골목상권,1000407,연서로33길,CS200007,치과의원,140020113,2150,...,0,1033,1117,32,280,324,402,458,654,1
6,2018,1,A,골목상권,1000407,연서로33길,CS200008,한의원,145038784,6360,...,20,2712,3648,10,391,896,1326,1930,1807,1


##### 코로나 이전과 이후의 조사범위를 동일하게 설정

코로나19 발생시점인 2020년 1월을 기준으로 2018년 1,2,3,4분기, 2019년 1,2,3,4분기 데이터와 2020년 1,2,3,4분기, 2021년 1,2분기 데이터만 수집된 상태.

그러므로 코로나 이전 데이터에서 2018년 1,2분기를 분석 범위에서 제외해야 코로나 전후에 대한 데이터 분포가 균형을 이루게 된다.

In [255]:
제외할index = 코로나이전df2.query("(기준_년_코드==2018 and (기준_분기_코드==1 or 기준_분기_코드==2))").index
코로나이전df3 = 코로나이전df2.drop(제외할index)
코로나이전df3.reset_index(drop=True, inplace=True)
#코로나이전df3.to_excel('코로나이전_매출데이터.xlsx', index=False)
코로나이전df3

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
0,2018,3,A,골목상권,1000094,마장로39길,CS200001,일반교습학원,182447499,644,...,25,291,308,0,5,39,464,92,0,5
1,2018,3,A,골목상권,1000066,원효로41길,CS100009,호프-간이주점,29160217,889,...,440,580,156,31,55,74,216,331,29,4
2,2018,3,A,골목상권,1000061,신흥로20길,CS300004,핸드폰,155314528,1128,...,0,627,448,0,197,56,416,189,216,1
3,2018,3,A,골목상권,1000878,주흥길,CS200029,네일숍,6170041,100,...,14,15,85,6,28,27,15,15,9,3
4,2018,3,A,골목상권,1000853,동광로12길,CS300033,철물점,68405170,569,...,0,276,113,0,25,59,99,147,59,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139603,2019,4,A,골목상권,1000440,세검정로1길,CS300014,신발,21965900,887,...,0,366,521,0,20,79,113,211,464,3
139604,2019,4,A,골목상권,1000910,도산대로81길,CS300022,화장품,1724533532,1824,...,382,782,896,17,377,630,293,218,144,7
139605,2019,4,A,골목상권,1000181,망우로21길,CS100004,양식음식점,562291801,23923,...,228,12935,9718,469,6166,6432,4792,3474,1319,2
139606,2019,4,A,골목상권,1000441,세무서2길,CS300002,편의점,518371562,81848,...,17654,57240,24608,2813,30887,15345,14338,12869,5596,2


#### 2) 코로나 이후 매출 데이터

In [256]:
코로나이후df = concat([df2020, df2021])
코로나이후df.reset_index(drop=True, inplace=True)

코로나이후df2 = 코로나이후df.query("상권_구분_코드_명 == '골목상권'")
#코로나이후df2.to_excel("코로나이후_매출데이터.xlsx", index=False)
코로나이후df2

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,시간대_건수~24_매출_건수,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수
2,2020,1,A,골목상권,1000022,창신2길,CS200001,일반교습학원,22367209.0,82,...,0,54,28,0,0,0,82,0,0,2
4,2020,1,A,골목상권,1000001,계동길,CS100001,한식음식점,431442455.0,18548,...,345,8752,7776,611,3828,4356,3428,2614,1690,19
5,2020,3,A,골목상권,1000127,광나루로12길,CS200016,당구장,424034.0,17,...,9,17,0,0,0,17,0,0,0,1
8,2020,1,A,골목상권,1000001,계동길,CS100002,중식음식점,25767161.0,1413,...,0,592,694,145,368,240,368,96,69,1
9,2020,1,A,골목상권,1000001,계동길,CS100003,일식음식점,55484866.0,1272,...,241,622,508,0,134,245,275,314,162,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197054,2021,1,A,골목상권,1000001,계동길,CS100005,제과점,165688274.0,18512,...,676,7523,9187,208,2471,4577,3857,2990,2609,4
197055,2021,1,A,골목상권,1000001,계동길,CS100004,양식음식점,199448496.0,4442,...,66,1778,2175,31,1505,1155,439,566,256,3
197056,2021,1,A,골목상권,1000001,계동길,CS100003,일식음식점,60325474.0,1563,...,136,519,635,0,185,339,242,279,112,2
197057,2021,1,A,골목상권,1000001,계동길,CS100002,중식음식점,39791852.0,2042,...,11,825,1041,243,629,307,322,269,97,1


기준_년_코드            0
기준_분기_코드           0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
                  ..
연령대_30_매출_건수       0
연령대_40_매출_건수       0
연령대_50_매출_건수       0
연령대_60_이상_매출_건수    0
점포수                0
Length: 80, dtype: int64

#### 3) 코로나 이전과 이후 데이터를 결합

##### 코로나 이전 데이터에 명목형 변수 추가

In [260]:
코로나이전df3['Covid19'] = 0

print(코로나이전df3.shape)

코로나이전df3.info()

(139608, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139608 entries, 0 to 139607
Data columns (total 81 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   기준_년_코드          139608 non-null  int64 
 1   기준_분기_코드         139608 non-null  int64 
 2   상권_구분_코드         139608 non-null  object
 3   상권_구분_코드_명       139608 non-null  object
 4   상권_코드            139608 non-null  int64 
 5   상권_코드_명          139608 non-null  object
 6   서비스_업종_코드        139608 non-null  object
 7   서비스_업종_코드_명      139608 non-null  object
 8   분기당_매출_금액        139608 non-null  int64 
 9   분기당_매출_건수        139608 non-null  int64 
 10  주중_매출_비율         139608 non-null  int64 
 11  주말_매출_비율         139608 non-null  int64 
 12  월요일_매출_비율        139608 non-null  int64 
 13  화요일_매출_비율        139608 non-null  int64 
 14  수요일_매출_비율        139608 non-null  int64 
 15  목요일_매출_비율        139608 non-null  int64 
 16  금요일_매출_비율        139608 non-null  int64 
 1

##### 코로나 이후 데이터에 명목형 변수 추가

In [ ]:
코로나이후df2['Covid19'] = 1

print(코로나이후df2.shape)

코로나이후df2.info()

(126320, 81)
<class 'pandas.core.frame.DataFrame'>
Index: 126320 entries, 2 to 197058
Data columns (total 81 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   기준_년_코드          126320 non-null  int64  
 1   기준_분기_코드         126320 non-null  int64  
 2   상권_구분_코드         126320 non-null  object 
 3   상권_구분_코드_명       126320 non-null  object 
 4   상권_코드            126320 non-null  int64  
 5   상권_코드_명          126320 non-null  object 
 6   서비스_업종_코드        126320 non-null  object 
 7   서비스_업종_코드_명      126320 non-null  object 
 8   분기당_매출_금액        126320 non-null  float64
 9   분기당_매출_건수        126320 non-null  int64  
 10  주중_매출_비율         126320 non-null  int64  
 11  주말_매출_비율         126320 non-null  int64  
 12  월요일_매출_비율        126320 non-null  int64  
 13  화요일_매출_비율        126320 non-null  int64  
 14  수요일_매출_비율        126320 non-null  int64  
 15  목요일_매출_비율        126320 non-null  int64  
 16  금요일_매출_비율        126320 non-nu

C:\Users\MegaIT302-02\AppData\Local\Temp\ipykernel_12004\74207771.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  코로나이후df2['Covid19'] = 1


In [263]:
코로나이후df2.isnull().sum()

기준_년_코드            0
기준_분기_코드           0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
                  ..
연령대_30_매출_건수       0
연령대_40_매출_건수       0
연령대_50_매출_건수       0
연령대_60_이상_매출_건수    0
점포수                0
Length: 80, dtype: int64

In [264]:
코로나이전df3.isnull().sum()

기준_년_코드            0
기준_분기_코드           0
상권_구분_코드           0
상권_구분_코드_명         0
상권_코드              0
                  ..
연령대_40_매출_건수       0
연령대_50_매출_건수       0
연령대_60_이상_매출_건수    0
점포수                0
Covid19            0
Length: 81, dtype: int64

##### 코로나 전후 매출데이터 병합

In [265]:
매출데이터_최종 = concat([코로나이전df3, 코로나이후df2])
매출데이터_최종.reset_index(drop=True, inplace=True)

print(매출데이터_최종.shape)

print(매출데이터_최종.info())

매출데이터_최종.head()

(265928, 81)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265928 entries, 0 to 265927
Data columns (total 81 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   기준_년_코드          265928 non-null  int64  
 1   기준_분기_코드         265928 non-null  int64  
 2   상권_구분_코드         265928 non-null  object 
 3   상권_구분_코드_명       265928 non-null  object 
 4   상권_코드            265928 non-null  int64  
 5   상권_코드_명          265928 non-null  object 
 6   서비스_업종_코드        265928 non-null  object 
 7   서비스_업종_코드_명      265928 non-null  object 
 8   분기당_매출_금액        265928 non-null  float64
 9   분기당_매출_건수        265928 non-null  int64  
 10  주중_매출_비율         265928 non-null  int64  
 11  주말_매출_비율         265928 non-null  int64  
 12  월요일_매출_비율        265928 non-null  int64  
 13  화요일_매출_비율        265928 non-null  int64  
 14  수요일_매출_비율        265928 non-null  int64  
 15  목요일_매출_비율        265928 non-null  int64  
 16  금요일_매출_비율        265928 n

,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,서비스_업종_코드,서비스_업종_코드_명,분기당_매출_금액,분기당_매출_건수,...,남성_매출_건수,여성_매출_건수,연령대_10_매출_건수,연령대_20_매출_건수,연령대_30_매출_건수,연령대_40_매출_건수,연령대_50_매출_건수,연령대_60_이상_매출_건수,점포수,Covid19
0,2018,3,A,골목상권,1000094,마장로39길,CS200001,일반교습학원,182447499.0,644,...,291,308,0,5,39,464,92,0,5,0.0
1,2018,3,A,골목상권,1000066,원효로41길,CS100009,호프-간이주점,29160217.0,889,...,580,156,31,55,74,216,331,29,4,0.0
2,2018,3,A,골목상권,1000061,신흥로20길,CS300004,핸드폰,155314528.0,1128,...,627,448,0,197,56,416,189,216,1,0.0
3,2018,3,A,골목상권,1000878,주흥길,CS200029,네일숍,6170041.0,100,...,15,85,6,28,27,15,15,9,3,0.0
4,2018,3,A,골목상권,1000853,동광로12길,CS300033,철물점,68405170.0,569,...,276,113,0,25,59,99,147,59,3,0.0


##### 분석에 사용할 요인만 추출 및 파생변수 생성

In [268]:
분석데이터 = 매출데이터_최종.filter(['기준_년_코드', '기준_분기_코드', 'Covid19', '상권_코드', '분기당_매출_금액', '주중_매출_금액', '주말_매출_금액', '시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '남성_매출_금액', '여성_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액', '점포수'], axis=1)

분석데이터['일과시간_금액'] = 분석데이터['시간대_06~11_매출_금액'] + 분석데이터['시간대_11~14_매출_금액'] + 분석데이터['시간대_14~17_매출_금액']

분석데이터['일과이후_금액'] = 분석데이터['시간대_17~21_매출_금액'] + 분석데이터['시간대_21~24_매출_금액'] + 분석데이터['시간대_00~06_매출_금액']

분석데이터['청년금액'] = (분석데이터['연령대_10_매출_금액'] + 분석데이터['연령대_20_매출_금액'] + 분석데이터['연령대_30_매출_금액'])
분석데이터['중년금액'] = (분석데이터['연령대_40_매출_금액'] + 분석데이터['연령대_50_매출_금액'])
분석데이터['장년금액'] = (분석데이터['연령대_60_이상_매출_금액'])

분석데이터.drop(['시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액', '연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액'], axis=1, inplace=True)

#분석데이터.to_excel('분석데이터.xlsx', index=False)

print(분석데이터.shape)

print(분석데이터.info())

분석데이터.isnull().sum()

(265928, 15)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265928 entries, 0 to 265927
Data columns (total 15 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   기준_년_코드    265928 non-null  int64  
 1   기준_분기_코드   265928 non-null  int64  
 2   Covid19    139608 non-null  float64
 3   상권_코드      265928 non-null  int64  
 4   분기당_매출_금액  265928 non-null  float64
 5   주중_매출_금액   265928 non-null  float64
 6   주말_매출_금액   265928 non-null  float64
 7   남성_매출_금액   265928 non-null  float64
 8   여성_매출_금액   265928 non-null  float64
 9   점포수        265928 non-null  int64  
 10  일과시간_금액    265928 non-null  float64
 11  일과이후_금액    265928 non-null  float64
 12  청년금액       265928 non-null  int64  
 13  중년금액       265928 non-null  float64
 14  장년금액       265928 non-null  int64  
dtypes: float64(9), int64(6)
memory usage: 30.4 MB
None


기준_년_코드           0
기준_분기_코드          0
Covid19      126320
상권_코드             0
분기당_매출_금액         0
주중_매출_금액          0
주말_매출_금액          0
남성_매출_금액          0
여성_매출_금액          0
점포수               0
일과시간_금액           0
일과이후_금액           0
청년금액              0
중년금액              0
장년금액              0
dtype: int64

### 4. 매출데이터에 상권영역 정보를 결합

#### 1) 결합할 변수만 추출

In [ ]:
tmp = df상권영역2.filter(['상권_코드','상권_코드_명','엑스좌표_값','와이좌표_값','시군구_코드','행정동_코드','시군구명','시군구코드','도심'])
tmp =  tmp.drop_duplicates()
tmp.dropna(inplace=True)
print(tmp.shape)
tmp.head()

(1496, 9)


,상권_코드,상권_코드_명,엑스좌표_값,와이좌표_값,시군구_코드,행정동_코드,시군구명,시군구코드,도심
0,1001453,낙성대시장,196121,442084,11620,11620585,관악구,11620,False
1,1001454,봉천제일종합시장,195147,442413,11620,11620595,관악구,11620,False
2,1001474,도곡시장,204551,444227,11680,11680650,강남구,11680,True
3,1001475,강남개포시장,206065,443310,11680,11680670,강남구,11680,True
4,1001412,화곡본동시장,186203,449328,11500,11500590,강서구,11500,False


#### 2) 매출데이터에 상권영역 정보 결합

In [269]:
상권매출df = merge(분석데이터, tmp, left_on="상권_코드", right_on="상권_코드", how="inner")

print(상권매출df.shape)

상권매출df.isnull().sum()

(265928, 23)


기준_년_코드           0
기준_분기_코드          0
Covid19      126320
상권_코드             0
분기당_매출_금액         0
주중_매출_금액          0
주말_매출_금액          0
남성_매출_금액          0
여성_매출_금액          0
점포수               0
일과시간_금액           0
일과이후_금액           0
청년금액              0
중년금액              0
장년금액              0
상권_코드_명           0
엑스좌표_값            0
와이좌표_값            0
시군구_코드            0
행정동_코드            0
시군구명              0
시군구코드             0
도심                0
dtype: int64

### 5. 인구 데이터

#### 1) 코로나 이전 배후지 직장 인구 데이터

##### 데이터 확인

In [ ]:
print(df직장인구2018.shape)

print(df직장인구2018.info())

df직장인구2018.head()

(4040, 27)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4040 entries, 0 to 4039
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   기준_년_코드              4040 non-null   int64 
 1   기준_분기_코드             4040 non-null   int64 
 2   상권_구분_코드             4040 non-null   object
 3   상권_구분_코드_명           4040 non-null   object
 4   상권_코드                4040 non-null   int64 
 5   상권_코드_명              4040 non-null   object
 6   총_직장_인구_수            4040 non-null   int64 
 7   남성_직장_인구_수           4040 non-null   int64 
 8   여성_직장_인구_수           4040 non-null   int64 
 9   연령대_10_직장_인구_수       4040 non-null   int64 
 10  연령대_20_직장_인구_수       4040 non-null   int64 
 11  연령대_30_직장_인구_수       4040 non-null   int64 
 12  연령대_40_직장_인구_수       4040 non-null   int64 
 13  연령대_50_직장_인구_수       4040 non-null   int64 
 14  연령대_60_이상_직장_인구_수    4040 non-null   int64 
 15  남성연령대_10_직장_인구_수     4040 non-null   int64 


,기준_년_코드,기준_분기_코드,상권_구분_코드,상권_구분_코드_명,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,연령대_10_직장_인구_수,...,남성연령대_30_직장_인구_수,남성연령대_40_직장_인구_수,남성연령대_50_직장_인구_수,남성연령대_60_이상_직장_인구_수,여성연령대_10_직장_인구_수,여성연령대_20_직장_인구_수,여성연령대_30_직장_인구_수,여성연령대_40_직장_인구_수,여성연령대_50_직장_인구_수,여성연령대_60_이상_직장_인구_수
0,2018,4,A,골목상권,1000159,자양로15길,6423,4133,2290,20,...,702,1241,1269,601,10,490,489,519,548,234
1,2018,4,A,골목상권,1000934,일원로9길,8245,2229,6016,2,...,727,537,385,229,2,1997,2093,1153,601,170
2,2018,4,A,골목상권,1000819,신림로48길,825,344,481,0,...,72,73,83,78,0,96,99,109,115,62
3,2018,4,A,골목상권,1000797,남부순환로161길,2836,1575,1261,2,...,364,437,354,309,0,197,311,300,308,145
4,2018,4,A,골목상권,1000692,대림로22길,2779,1349,1430,9,...,218,314,330,384,6,294,271,329,326,204


In [ ]:
코로나이전_직장인구_df = concat([df직장인구2018, df직장인구2019])
코로나이전_직장인구_df.reset_index(drop=True, inplace=True)
drop_index = 코로나이전_직장인구_df.query("기준_년_코드==2018 and (기준_분기_코드==1 or 기준_분기_코드==2)").index
코로나이전_직장인구_df.drop(drop_index, inplace=True)
코로나이전_직장인구_df = 코로나이전_직장인구_df.query('상권_구분_코드_명 == "골목상권"')
코로나이전_직장인구_df.reset_index(drop=True, inplace=True)
코로나이전_직장인구df = 코로나이전_직장인구_df.filter(['기준_년_코드', '기준_분기_코드', '상권_코드', '총_직장_인구_수'])
#코로나이전_직장인구df.head()

코로나이전_직장인구df.groupby(['기준_년_코드', '기준_분기_코드']).count()

상권_코드  총_직장_인구_수
기준_년_코드 기준_분기_코드                  
2018    3          1010       1010
        4          1010       1010
2019    1          1010       1010
        2          1010       1010
        3          1010       1010
        4          1010       1010

#### 2) 코로나 이후 배후지 직장 인구 데이터

In [ ]:
코로나이후_직장인구_df = concat([df직장인구2020, df직장인구2021])
코로나이전_직장인구_df.reset_index(drop=True, inplace=True)
코로나이후_직장인구_df = 코로나이후_직장인구_df.query('상권_구분_코드_명 == "골목상권"')
코로나이후_직장인구_df.reset_index(drop=True, inplace=True)
코로나이후_직장인구df = 코로나이후_직장인구_df.filter(['기준_년_코드', '기준_분기_코드', '상권_코드', '총_직장_인구_수'])
코로나이후_직장인구df.head()

,기준_년_코드,기준_분기_코드,상권_코드,총_직장_인구_수
0,2020,4,1000488,3946
1,2020,4,1000995,1824
2,2020,4,1000994,2045
3,2020,4,1000208,1507
4,2020,4,1000993,21487


#### 3) 코로나 이전 배후지 생활 인구 데이터

##### 데이터 확인

In [ ]:
df생활인구2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4040 entries, 0 to 4039
Columns: 532 entries, 기준_년_코드 to 여성연령대_60_이상_일요일시간대_6_생활인구_수
dtypes: int64(529), object(3)
memory usage: 16.4+ MB


##### 데이터 병합 및 추출

In [ ]:
코로나이전_생활인구_df = concat([df생활인구2018, df생활인구2019])
코로나이전_생활인구_df.reset_index(drop=True, inplace=True)
drop_index = 코로나이전_생활인구_df.query("(기준_년_코드==2018 and (기준_분기_코드==1 or 기준_분기_코드==2))").index
코로나이전_생활인구_df.drop(drop_index, inplace=True)
코로나이전_생활인구_df = 코로나이전_생활인구_df.query('상권_구분_코드_명 == "골목상권"')
코로나이전_생활인구_df.reset_index(drop=True, inplace=True)
코로나이전_생활인구df = 코로나이전_생활인구_df.filter(['기준_년_코드', '기준_분기_코드', '상권_코드', '총_생활인구_수'])
코로나이전_생활인구df.head()

,기준_년_코드,기준_분기_코드,상권_코드,총_생활인구_수
0,2018,4,1000005,53242
1,2018,4,1000107,752834
2,2018,4,1000403,2007766
3,2018,4,1000094,775804
4,2018,4,1000650,1349770


#### 5) 코로나 이후 배후지 생활 인구 데이터

In [ ]:
코로나이후_생활인구_df = concat([df생활인구2020, df생활인구2021])
코로나이후_생활인구_df.reset_index(drop=True, inplace=True)
코로나이후_생활인구_df = 코로나이후_생활인구_df.query('상권_구분_코드_명 == "골목상권"')
코로나이후_생활인구_df.reset_index(drop=True, inplace=True)
코로나이후_생활인구df = 코로나이후_생활인구_df.filter(['기준_년_코드', '기준_분기_코드', '상권_코드', '총_생활인구_수'])
코로나이후_생활인구df.head()

,기준_년_코드,기준_분기_코드,상권_코드,총_생활인구_수
0,2020,4,1000117,4561311
1,2020,4,1000524,660691
2,2020,4,1000101,5400680
3,2020,4,1000777,4587594
4,2020,4,1000530,3339989


#### 6) 인구 데이터 결합

##### 코로나 전후 직장인구 데이터 결합

In [ ]:
직장인구 = concat([코로나이전_직장인구df, 코로나이후_직장인구df])

print(직장인구.shape)

print(직장인구.info())

직장인구.head()

(12120, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 12120 entries, 0 to 6059
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   기준_년_코드    12120 non-null  int64
 1   기준_분기_코드   12120 non-null  int64
 2   상권_코드      12120 non-null  int64
 3   총_직장_인구_수  12120 non-null  int64
dtypes: int64(4)
memory usage: 473.4 KB
None


,기준_년_코드,기준_분기_코드,상권_코드,총_직장_인구_수
0,2018,4,1000159,6423
1,2018,4,1000934,8245
2,2018,4,1000819,825
3,2018,4,1000797,2836
4,2018,4,1000692,2779


In [ ]:
직장인구.groupby(['기준_년_코드', '기준_분기_코드']).count()

상권_코드  총_직장_인구_수
기준_년_코드 기준_분기_코드                  
2018    3          1010       1010
        4          1010       1010
2019    1          1010       1010
        2          1010       1010
        3          1010       1010
        4          1010       1010
2020    1          1010       1010
        2          1010       1010
        3          1010       1010
        4          1010       1010
2021    1          1010       1010
        2          1010       1010

##### 코로나 전후 생활인구 데이터 결합

In [276]:
생활인구 = concat([코로나이전_생활인구df, 코로나이후_생활인구df])

print(생활인구.shape)

print(생활인구.info())

생활인구.head()
생활인구.isnull().sum()

(12120, 4)
<class 'pandas.core.frame.DataFrame'>
Index: 12120 entries, 0 to 6059
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   기준_년_코드   12120 non-null  int64
 1   기준_분기_코드  12120 non-null  int64
 2   상권_코드     12120 non-null  int64
 3   총_생활인구_수  12120 non-null  int64
dtypes: int64(4)
memory usage: 473.4 KB
None


기준_년_코드     0
기준_분기_코드    0
상권_코드       0
총_생활인구_수    0
dtype: int64

##### 직장인구 + 생활인구

In [277]:
인구데이터 = merge(직장인구, 생활인구, left_on=['기준_년_코드','기준_분기_코드','상권_코드'], right_on=['기준_년_코드','기준_분기_코드','상권_코드'], how='outer')

print(인구데이터.shape)

print(인구데이터.info())

#인구데이터.to_excel('인구데이터.xlsx', index=False)
인구데이터.isnull().sum()

(12120, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12120 entries, 0 to 12119
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   기준_년_코드    12120 non-null  int64
 1   기준_분기_코드   12120 non-null  int64
 2   상권_코드      12120 non-null  int64
 3   총_직장_인구_수  12120 non-null  int64
 4   총_생활인구_수   12120 non-null  int64
dtypes: int64(5)
memory usage: 473.6 KB
None


기준_년_코드      0
기준_분기_코드     0
상권_코드        0
총_직장_인구_수    0
총_생활인구_수     0
dtype: int64

### 6. 최종 데이터 프레임 구성

`(매출데이터 + 상권데이터) + 인구데이터`

In [ ]:
상권매출df
상권매출df.to_excel("상권매출df.xlsx", index=False)

In [282]:
전처리완료df = merge(상권매출df, 인구데이터, left_on=['기준_년_코드','기준_분기_코드','상권_코드'], right_on=['기준_년_코드','기준_분기_코드','상권_코드'], how='outer')

print(전처리완료df.shape)

print(전처리완료df.info())

전처리완료df.to_excel("분석용데이터.xlsx", index=False)
전처리완료df.isnull().sum()

(265937, 25)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 265937 entries, 0 to 265936
Data columns (total 25 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   기준_년_코드    265937 non-null  int64  
 1   기준_분기_코드   265937 non-null  int64  
 2   Covid19    139608 non-null  float64
 3   상권_코드      265937 non-null  int64  
 4   분기당_매출_금액  265928 non-null  float64
 5   주중_매출_금액   265928 non-null  float64
 6   주말_매출_금액   265928 non-null  float64
 7   남성_매출_금액   265928 non-null  float64
 8   여성_매출_금액   265928 non-null  float64
 9   점포수        265928 non-null  float64
 10  일과시간_금액    265928 non-null  float64
 11  일과이후_금액    265928 non-null  float64
 12  청년금액       265928 non-null  float64
 13  중년금액       265928 non-null  float64
 14  장년금액       265928 non-null  float64
 15  상권_코드_명    265928 non-null  object 
 16  엑스좌표_값     265928 non-null  float64
 17  와이좌표_값     265928 non-null  float64
 18  시군구_코드     265928 non-null  float64
 19  행정동_코드    

기준_년_코드           0
기준_분기_코드          0
Covid19      126329
상권_코드             0
분기당_매출_금액         9
주중_매출_금액          9
주말_매출_금액          9
남성_매출_금액          9
여성_매출_금액          9
점포수               9
일과시간_금액           9
일과이후_금액           9
청년금액              9
중년금액              9
장년금액              9
상권_코드_명           9
엑스좌표_값            9
와이좌표_값            9
시군구_코드            9
행정동_코드            9
시군구명              9
시군구코드             9
도심                9
총_직장_인구_수         0
총_생활인구_수          0
dtype: int64